# GNN Evaluation

This notebook evaluates the trained GNN model on unseen test data.
- Loads the best model checkpoint from training
- Runs it on the test set: for each hour, predicts next-hour ridership for all stations
- Compares predictions to ground truth, both in normalized and real space
- Reports overall error, error by hour, and per-station breakdown
- See which stations are easiest/hardest to predict

In [57]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch_geometric.nn import SAGEConv
from tqdm import tqdm

## Config

In [58]:
SPLIT = "test"
HIDDEN_DIM = 64

# Paths
ROOT = os.path.dirname(os.path.abspath(""))
PROC_DIR = os.path.join(ROOT, "data", "processed")
MODEL_PATH = os.path.join(ROOT, "models", "model.pt")
STATS_PATH = os.path.join(PROC_DIR, "stats.csv")
CMPLX_PATH = os.path.join(PROC_DIR, "ComplexNodes.csv")
EDGES_PATH = os.path.join(PROC_DIR, "ComplexEdges.csv")

## Model Definition

Match the model architecture used in training

In [59]:
class DirSAGEEmbRes(nn.Module):
    def __init__(self, num_nodes: int, in_dim: int, hidden_dim: int, emb_dim: int = 16):
        super().__init__()
        self.node_emb = nn.Embedding(num_nodes, emb_dim)
        d0 = in_dim + emb_dim

        self.in1 = SAGEConv(d0, hidden_dim)
        self.in2 = SAGEConv(hidden_dim, hidden_dim)

        self.out1 = SAGEConv(d0, hidden_dim)
        self.out2 = SAGEConv(hidden_dim, hidden_dim)

        self.lin = nn.Linear(2 * hidden_dim, 1)

    def forward(self, x, edge_in, edge_out):
        node_ids = torch.arange(x.size(0), device=x.device)
        x = torch.cat([x, self.node_emb(node_ids)], dim=1)

        h_in1 = torch.relu(self.in1(x, edge_in))
        h_in2 = torch.relu(self.in2(h_in1, edge_in))
        h_in  = h_in2 + h_in1

        h_out1 = torch.relu(self.out1(x, edge_out))
        h_out2 = torch.relu(self.out2(h_out1, edge_out))
        h_out  = h_out2 + h_out1

        h = torch.cat([h_in, h_out], dim=-1)
        return self.lin(h).squeeze(-1)

## Load Graph Structure & Stats

- **Node mapping**: maps station complex IDs to node indices
- **Edges**: pairs of connected stations
- **Stats**: per-station mean and std, used to denormalize predictions back to real counts

In [60]:
# Load node mapping
cmplx_df = pd.read_csv(CMPLX_PATH)
ComplexNodes = dict(zip(cmplx_df["complex_id"], cmplx_df["node_id"]))
node_to_cmplx = dict(zip(cmplx_df["node_id"], cmplx_df["complex_id"]))

num_nodes = len(ComplexNodes)

# Load edges
edges_df = pd.read_csv(EDGES_PATH)

edge_in_list = []   # from -> to
edge_out_list = []  # to -> from

for _, row in edges_df.iterrows():
    s, e = row["from_complex_id"], row["to_complex_id"]
    if s in ComplexNodes and e in ComplexNodes:
        u, v = ComplexNodes[s], ComplexNodes[e]
        edge_in_list.append([u, v])
        edge_out_list.append([v, u])

# Add self-loops
for i in range(num_nodes):
    edge_in_list.append([i, i])
    edge_out_list.append([i, i])

edge_in = torch.tensor(edge_in_list, dtype=torch.long).T
edge_out = torch.tensor(edge_out_list, dtype=torch.long).T

# Load per-station normalization stats
stats = pd.read_csv(STATS_PATH)
stn_mean = dict(zip(stats["station_complex_id"], stats["mean"]))
stn_std = dict(zip(stats["station_complex_id"], stats["std"]))

print(f"Nodes: {num_nodes}, Edge_in: {edge_in.shape[1]}, Edge_out: {edge_out.shape[1]}")

Nodes: 424, Edge_in: 976, Edge_out: 976


## Load Model

Loads the best model checkpoint from training.

In [61]:
model = DirSAGEEmbRes(num_nodes=num_nodes, in_dim=5, hidden_dim=HIDDEN_DIM, emb_dim=16)
model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))
model.eval()
print(f"Loaded model from {MODEL_PATH}")

Loaded model from c:\Users\setho\PersonalProjects\hush\models\model.pt


## Load Split Data & Build Snapshots

Loads the test file. Each row is one station's normalised ridership at one timestamp, with time encodings.

Build graph snapshots: for every pair of consecutive hours, the model sees hour $t$ and must predict hour $t+1$ for all stations.

In [62]:
# Load test split
split_path = os.path.join(PROC_DIR, f"{SPLIT}.parquet")
df = pd.read_parquet(split_path)
print(f"{SPLIT.upper()} set: {len(df):,} rows")
print(f"Date range: {df['transit_timestamp'].min()} -> {df['transit_timestamp'].max()}")

# timestamp -> dataframe containing all stations observed at that timestamp
groups = {t: g for t, g in df.groupby("transit_timestamp")}
timestamps = sorted(groups.keys())

"""
Build one entry per snapshot:
- features   : X at time t0
- targets    : y at time t1 (normalised ridership)
- raw_targets: y_raw at time t1 (actual ridership)
- masks      : m (boolean mask), which nodes actually have a target at t1
- hours      : hour of day for t1
"""
features, targets, raw_targets, masks, hours = [], [], [], [], []

for t0, t1 in tqdm(zip(timestamps[:-1], timestamps[1:]), total=len(timestamps)-1, desc="Building snapshots"):
    if (t1 - t0).total_seconds() > 3600:
        continue
    
    # Rows for time t0 and t1
    g0 = groups[t0]
    g1 = groups[t1]

    # Allocate dense, node-aligned tensors
    X = torch.zeros(num_nodes, 5, dtype=torch.float32)
    y = torch.zeros(num_nodes, dtype=torch.float32)
    y_raw = torch.zeros(num_nodes, dtype=torch.float32)
    m = torch.zeros(num_nodes, dtype=torch.bool)

    # Node indices at each timestamp
    idx0 = torch.tensor(g0["node_id"].values, dtype=torch.long)
    idx1 = torch.tensor(g1["node_id"].values, dtype=torch.long)

    # For each node at time t0, write its features to the correct row
    X[idx0, 0] = torch.tensor(g0["ridership_norm"].values, dtype=torch.float32)
    X[idx0, 1] = torch.tensor(g0["sin_hour"].values, dtype=torch.float32)
    X[idx0, 2] = torch.tensor(g0["cos_hour"].values, dtype=torch.float32)
    X[idx0, 3] = torch.tensor(g0["sin_dow"].values, dtype=torch.float32)
    X[idx0, 4] = torch.tensor(g0["cos_dow"].values, dtype=torch.float32)

    # Fill targets for nodes present at t1, build mask, avoid scoring on fake zeros
    y[idx1] = torch.tensor(g1["ridership_norm"].values, dtype=torch.float32)
    y_raw[idx1] = torch.tensor(g1["ridership"].values, dtype=torch.float32)
    m[idx1] = True

    # Store snapshot
    features.append(X)
    targets.append(y)
    raw_targets.append(y_raw)
    masks.append(m)
    hours.append(pd.Timestamp(t1).hour)

print(f"Snapshots: {len(features)}")
del df, groups

TEST set: 3,649,611 rows
Date range: 2024-01-01 00:00:00 -> 2024-12-31 23:00:00


Building snapshots: 100%|██████████| 8782/8782 [00:07<00:00, 1150.09it/s]

Snapshots: 8781


In [63]:
# For each snapshot, run the model, collect predictions and ground truth
all_pred_norm, all_true_norm = [], [] # Normalised round truth 
all_pred_raw, all_true_raw = [], []   # Raw ground truth
all_hours, all_stations = [], []      # Metadata for slicing results by hour/station

# Disable gradient calculation to avoid accidental backpropagation
with torch.no_grad():
    # Node features, normalised targets, raw targets, mask (which nodes have a target), hour
    for X, y_norm, y_raw, m, hour in tqdm(
        zip(features, targets, raw_targets, masks, hours),
        total=len(features),
        desc="Evaluating",
    ):
        # y          -> ground truth
        # y_hat      -> estimate of y made by model
        # y_hat_norm -> normalised estimate of y made by model 
        # Directed dual-pass model
        y_hat_norm = model(X, edge_in, edge_out)

        idx = torch.where(m)[0]  # Only score nodes with a valid target at t+1

        for node_id in idx.tolist():
            # Ignore nodes that have no real complex mapping
            if node_id not in node_to_cmplx:
                continue
            
            # Ground truth and prediction for this node
            cmplx_id = node_to_cmplx[node_id]
            true_norm = y_norm[node_id].item()
            pred_norm = y_hat_norm[node_id].item()
            true_raw_val = y_raw[node_id].item()

            # Un-normalise prediction back to raw units
            mean = stn_mean.get(cmplx_id, 0)
            std = stn_std.get(cmplx_id, 1)
            pred_raw = pred_norm * std + mean

            # Store results and clamp negative predictions
            all_pred_norm.append(pred_norm)
            all_true_norm.append(true_norm)
            all_pred_raw.append(max(0, pred_raw))
            all_true_raw.append(true_raw_val)
            all_hours.append(hour)
            all_stations.append(cmplx_id)


pred_norm = np.array(all_pred_norm)
true_norm = np.array(all_true_norm)
pred_raw = np.array(all_pred_raw)
true_raw = np.array(all_true_raw)
hours_arr = np.array(all_hours)
stations_arr = np.array(all_stations)

# Number of node-level evaluations after ignoring nodes without targets and nodes without complex mappings
print(f"Total predictions (masked): {len(pred_raw):,}")

Evaluating: 100%|██████████| 8781/8781 [01:19<00:00, 110.35it/s]


Total predictions (masked): 3,648,795


## Overall Metrics

Calculates and prints:
- **MSE/MAE in normalized space**
- **MSE/MAE/RMSE in raw space**
- **Median absolute error**
- **R^2 score**

MSE/MAE in the normalised space is measured in z-score units and MSE/MAE/RMSE in the raw space is measured in terms of the real number tap-ins.

MedAE is very robust ot outliers. If MAE >> MedAE, this may indicate that there are rare but big misses in the model's predictions.

R^2 score measures how much better we are than a dumb baseline like predicting the mean of true_raw. It measures how well the model explains variance.
- 1.0 = perfect
- 0.0 = no better than predicting the mean
- < 0 = worse than predicting the mean 

This gives a sense of both relative and absolute model accuracy.

In [64]:
# --- Normalized space ---
mse_norm = np.mean((pred_norm - true_norm) ** 2)
mae_norm = np.mean(np.abs(pred_norm - true_norm))

# --- Raw space ---
mse_raw = np.mean((pred_raw - true_raw) ** 2)
mae_raw = np.mean(np.abs(pred_raw - true_raw))
rmse_raw = np.sqrt(mse_raw)

# --- R² score ---
ss_res = np.sum((true_raw - pred_raw) ** 2)
ss_tot = np.sum((true_raw - np.mean(true_raw)) ** 2)
r2 = 1 - (ss_res / ss_tot) if ss_tot > 0 else 0

# --- Median absolute error ---
median_ae = np.median(np.abs(pred_raw - true_raw))

print("=" * 50)
print("OVERALL METRICS")
print("=" * 50)
print(f"\n  Normalized space:")
print(f"    MSE  = {mse_norm:.4f}")
print(f"    MAE  = {mae_norm:.4f}")
print(f"\n  Raw space:")
print(f"    MSE   = {mse_raw:.2f}")
print(f"    RMSE  = {rmse_raw:.2f}")
print(f"    MAE   = {mae_raw:.2f} tap-ins")
print(f"    MedAE = {median_ae:.2f} tap-ins")
print(f"    R^2    = {r2:.4f}")

OVERALL METRICS

  Normalized space:
    MSE  = 0.1216
    MAE  = 0.2011

  Raw space:
    MSE   = 18685.71
    RMSE  = 136.70
    MAE   = 48.18 tap-ins
    MedAE = 17.73 tap-ins
    R^2    = 0.9623


## Error by Hour of Day

Shows how model error varies by time of day. This helps us to identify problems like whether or not hte model struggles to predict rush hour traffic correctly.

For each hour, prints the MAE and a bar chart for quick visual comparison.

In [65]:
print(f"{'Hour':>6}  {'MAE':>8}  {'Count':>8}  Bar")
print("-" * 45)
for h in range(24):
    mask = hours_arr == h
    if mask.sum() == 0:
        continue
    h_mae = np.mean(np.abs(pred_raw[mask] - true_raw[mask]))
    h_count = mask.sum()
    bar = "█" * int(h_mae / 5)
    print(f"  {h:02d}:00  {h_mae:>8.2f}  {h_count:>8}  {bar}")

  Hour       MAE     Count  Bar
---------------------------------------------
  00:00     21.65    149280  ████
  01:00     18.88    146720  ███
  02:00     22.57    145604  ████
  03:00     16.39    147719  ███
  04:00     18.00    151917  ███
  05:00     31.06    152980  ██████
  06:00     46.81    153202  █████████
  07:00     88.29    153251  █████████████████
  08:00    117.45    153164  ███████████████████████
  09:00     51.13    153139  ██████████
  10:00     43.50    153117  ████████
  11:00     38.76    153082  ███████
  12:00     39.73    153107  ███████
  13:00     42.82    153168  ████████
  14:00     51.25    153247  ██████████
  15:00     64.43    153254  ████████████
  16:00     67.14    153210  █████████████
  17:00    117.54    153154  ███████████████████████
  18:00     59.61    153140  ███████████
  19:00     42.46    153044  ████████
  20:00     37.03    152967  ███████
  21:00     38.79    152888  ███████
  22:00     39.48    152579  ███████
  23:00     36.80    1

## Per-Station Breakdown

For each station, computes:
- MAE
- Average true ridership
- MAPE (mean absolute percentage error)
- Number of predictions

Prints the 10 worst and 10 best stations by MAE.

In [66]:
station_errors = {}
for cmplx_id in np.unique(stations_arr):
    mask = stations_arr == cmplx_id
    if mask.sum() < 5:
        continue
    s_mae = np.mean(np.abs(pred_raw[mask] - true_raw[mask]))
    s_avg_ridership = np.mean(true_raw[mask])
    station_errors[cmplx_id] = {
        "mae": s_mae,
        "avg_ridership": s_avg_ridership,
        "mape": (s_mae / (s_avg_ridership + 1e-6)) * 100,
        "n": int(mask.sum()),
    }

sorted_stations = sorted(station_errors.items(), key=lambda x: x[1]["mae"], reverse=True)

print("Top 10 WORST stations (highest MAE):")
print(f"  {'Station':>10}  {'MAE':>8}  {'Avg Ridership':>14}  {'MAPE%':>7}  {'n':>6}")
print(f"  {'-'*10}  {'-'*8}  {'-'*14}  {'-'*7}  {'-'*6}")
for cmplx_id, err in sorted_stations[:10]:
    print(f"  {cmplx_id:>10}  {err['mae']:>8.2f}  {err['avg_ridership']:>14.2f}  {err['mape']:>6.1f}%  {err['n']:>6}")

print("Top 10 BEST stations (lowest MAE):")
print(f"  {'Station':>10}  {'MAE':>8}  {'Avg Ridership':>14}  {'MAPE%':>7}  {'n':>6}")
print(f"  {'-'*10}  {'-'*8}  {'-'*14}  {'-'*7}  {'-'*6}")
for cmplx_id, err in sorted_stations[-10:]:
    print(f"  {cmplx_id:>10}  {err['mae']:>8.2f}  {err['avg_ridership']:>14.2f}  {err['mape']:>6.1f}%  {err['n']:>6}")

Top 10 WORST stations (highest MAE):
     Station       MAE   Avg Ridership    MAPE%       n
  ----------  --------  --------------  -------  ------
         611    617.97         5231.56    11.8%    8781
         610    542.37         3869.82    14.0%    8781
         607    346.41         2855.46    12.1%    8781
         628    302.45         2198.59    13.8%    8781
         164    287.34         2126.29    13.5%    8781
         602    283.82         2613.00    10.9%    8781
         318    271.00         1896.79    14.3%    8781
         225    242.33         1425.44    17.0%    8781
         624    214.74         1525.03    14.1%    8781
         614    212.01         1935.46    11.0%    8781
Top 10 BEST stations (lowest MAE):
     Station       MAE   Avg Ridership    MAPE%       n
  ----------  --------  --------------  -------  ------
         205      7.27           39.37    18.5%    8604
         200      6.93           22.36    31.0%    8295
         196      6.29          

In [67]:
print("Top 10 WORST stations (highest MAE):")
print(f"  {'Station':>10}  {'MAE':>8}  {'Avg Ridership':>14}  {'MAPE%':>7}  {'n':>6}")
print(f"  {'-'*10}  {'-'*8}  {'-'*14}  {'-'*7}  {'-'*6}")
for cmplx_id, err in sorted_stations[:10]:
    print(f"  {cmplx_id:>10}  {err['mae']:>8.2f}  {err['avg_ridership']:>14.2f}  {err['mape']:>6.1f}%  {err['n']:>6}")

Top 10 WORST stations (highest MAE):
     Station       MAE   Avg Ridership    MAPE%       n
  ----------  --------  --------------  -------  ------
         611    617.97         5231.56    11.8%    8781
         610    542.37         3869.82    14.0%    8781
         607    346.41         2855.46    12.1%    8781
         628    302.45         2198.59    13.8%    8781
         164    287.34         2126.29    13.5%    8781
         602    283.82         2613.00    10.9%    8781
         318    271.00         1896.79    14.3%    8781
         225    242.33         1425.44    17.0%    8781
         624    214.74         1525.03    14.1%    8781
         614    212.01         1935.46    11.0%    8781


In [68]:
print("Top 10 BEST stations (lowest MAE):")
print(f"  {'Station':>10}  {'MAE':>8}  {'Avg Ridership':>14}  {'MAPE%':>7}  {'n':>6}")
print(f"  {'-'*10}  {'-'*8}  {'-'*14}  {'-'*7}  {'-'*6}")
for cmplx_id, err in sorted_stations[-10:]:
    print(f"  {cmplx_id:>10}  {err['mae']:>8.2f}  {err['avg_ridership']:>14.2f}  {err['mape']:>6.1f}%  {err['n']:>6}")

Top 10 BEST stations (lowest MAE):
     Station       MAE   Avg Ridership    MAPE%       n
  ----------  --------  --------------  -------  ------
         205      7.27           39.37    18.5%    8604
         200      6.93           22.36    31.0%    8295
         196      6.29           20.95    30.0%    8457
         197      5.16           21.18    24.3%    8276
         207      5.11           20.23    25.3%    8118
         203      4.94           19.89    24.9%    8590
         202      4.88           10.23    47.8%    7188
         206      4.80           18.10    26.5%    8469
         201      4.73           13.01    36.4%    7917
         199      3.11            8.21    37.9%    7717


## MAPE Distribution

In [69]:
mapes = [v["mape"] for v in station_errors.values()]
print(f"MAPE across stations:")
print(f"  Median = {np.median(mapes):.1f}%")
print(f"  Mean   = {np.mean(mapes):.1f}%")
print(f"  25th   = {np.percentile(mapes, 25):.1f}%")
print(f"  75th   = {np.percentile(mapes, 75):.1f}%")

MAPE across stations:
  Median = 15.5%
  Mean   = 16.6%
  25th   = 13.9%
  75th   = 17.7%


## MAPE Distribution

Shows the distribution of mean absolute percentage error (MAPE) across all stations. Useful for understanding typical vs. worst-case error.

In [70]:
mapes = [v["mape"] for v in station_errors.values()]
print(f"MAPE across stations:")
print(f"  Median = {np.median(mapes):.1f}%")
print(f"  Mean   = {np.mean(mapes):.1f}%")
print(f"  25th   = {np.percentile(mapes, 25):.1f}%")
print(f"  75th   = {np.percentile(mapes, 75):.1f}%")

MAPE across stations:
  Median = 15.5%
  Mean   = 16.6%
  25th   = 13.9%
  75th   = 17.7%


In [71]:
import numpy as np
import pandas as pd

# --- EDIT THESE NAMES ONLY IF YOUR NOTEBOOK USES DIFFERENT ONES ---
y_true = true_raw
y_pred = pred_raw
station_ids = stations_arr
# -----------------------------------------------------------------

# Safety: drop NaNs/infs if any
mask_ok = np.isfinite(y_true) & np.isfinite(y_pred)
y_true = y_true[mask_ok]
y_pred = y_pred[mask_ok]
station_ids = station_ids[mask_ok]

# ---- Overall metrics (system-wide) ----
mae = np.mean(np.abs(y_true - y_pred))
rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))
wmape = np.sum(np.abs(y_true - y_pred)) / max(np.sum(np.abs(y_true)), 1e-8)   # weighted MAPE

print("\n=== Overall (masked) metrics ===")
print(f"MAE   = {mae:,.2f}")
print(f"RMSE  = {rmse:,.2f}")
print(f"WMAPE = {wmape*100:.2f}%")

# ---- Baseline: predict station mean ----
# Build baseline predictions aligned with each (station, time) point
baseline = np.array([stn_mean.get(int(s), np.nan) for s in station_ids], dtype=float)

mask_b = np.isfinite(baseline)
y_true_b = y_true[mask_b]
y_pred_b = baseline[mask_b]

mae_b = np.mean(np.abs(y_true_b - y_pred_b))
wmape_b = np.sum(np.abs(y_true_b - y_pred_b)) / max(np.sum(np.abs(y_true_b)), 1e-8)

print("\n=== Baseline: station mean ===")
print(f"MAE   = {mae_b:,.2f}")
print(f"WMAPE = {wmape_b*100:.2f}%")

print("\n=== Improvement vs baseline (station mean) ===")
print(f"ΔMAE   = {mae_b - mae:,.2f} (positive means model is better)")
print(f"ΔWMAPE = {(wmape_b - wmape)*100:.2f} pp (positive means model is better)")

# ---- Optional: per-station WMAPE leaderboard (fairer than MAE-only) ----
df_cmp = pd.DataFrame({
    "Station": station_ids.astype(int),
    "y_true": y_true,
    "y_pred": y_pred,
})
df_cmp["abs_err"] = (df_cmp["y_true"] - df_cmp["y_pred"]).abs()

per_station_wmape = (
    df_cmp.groupby("Station")
    .agg(
        WMAPE=("abs_err", lambda s: s.sum() / max(df_cmp.loc[s.index, "y_true"].abs().sum(), 1e-8)),
        MAE=("abs_err", "mean"),
        AvgRidership=("y_true", "mean"),
        n=("y_true", "size"),
    )
    .reset_index()
)
per_station_wmape["WMAPE%"] = per_station_wmape["WMAPE"] * 100
per_station_wmape = per_station_wmape.drop(columns=["WMAPE"])

print("\nTop 10 WORST stations by WMAPE%:")
display(per_station_wmape.sort_values("WMAPE%", ascending=False).head(10))

print("\nTop 10 BEST stations by WMAPE%:")
display(per_station_wmape.sort_values("WMAPE%", ascending=True).head(10))


=== Overall (masked) metrics ===
MAE   = 48.18
RMSE  = 136.70
WMAPE = 14.58%

=== Baseline: station mean ===
MAE   = 235.27
WMAPE = 71.18%

=== Improvement vs baseline (station mean) ===
ΔMAE   = 187.09 (positive means model is better)
ΔWMAPE = 56.60 pp (positive means model is better)

Top 10 WORST stations by WMAPE%:


,Station,MAE,AvgRidership,n,WMAPE%
373,448,152.334252,230.366196,8774,66.126999
161,202,4.884463,10.226349,7188,47.763509
69,85,26.071011,58.726673,7727,44.393816
67,83,30.791106,71.428374,7979,43.107667
158,199,3.110756,8.212518,7717,37.878224
160,201,4.728581,13.006063,7917,36.356741
70,86,12.441631,36.626654,7556,33.968789
311,374,10.147177,32.294203,8314,31.421049
159,200,6.928390,22.361905,8295,30.983003
110,138,19.652060,63.511918,8726,30.942318



Top 10 BEST stations by WMAPE%:


,Station,MAE,AvgRidership,n,WMAPE%
216,264,48.766114,483.946134,8781,10.076765
414,623,127.019270,1258.651976,8781,10.091691
372,447,173.204917,1684.875271,8755,10.279985
215,263,36.911824,347.711731,8780,10.615639
393,602,283.820120,2613.003303,8781,10.861835
404,613,148.439435,1365.809817,8781,10.868236
392,601,131.112705,1203.958433,8781,10.890136
376,451,81.280070,743.927571,8781,10.925804
405,614,212.009474,1935.461793,8781,10.953948
249,303,29.050506,264.061714,8507,11.001408
